In [141]:
import os
import json
import pandas as pd
import traceback
from dotenv import load_dotenv
import cohere
load_dotenv()

True

In [142]:
cohere_api_key=os.environ['COHERE_API_KEY']

In [143]:
co = cohere.ClientV2(api_key=cohere_api_key)


In [144]:
from langchain_cohere import ChatCohere

llm = ChatCohere(model="command-r")

In [145]:
from langchain.llms import Cohere
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import PromptLayerCallbackHandler
import PyPDF2

In [165]:
RESPONSE_JSON={
    "1":{
        "mcq":"multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here"
        },
        "correct":"correct answer",
    },
    "2":{
        "mcq":"multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here"
        },
        "correct":"correct answer",
    },
    "3":{
        "mcq":"multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here"
        },
        "correct":"correct answer",
    },
    "4":{
        "mcq":"multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here"
        },
        "correct":"correct answer",
    },
    "5":{
        "mcq":"multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here"
        },
        "correct":"correct answer",
    }
}

In [166]:
TEMPLATE="""
    Text:{text}
    You are an expert MCQ maker.Given the above text it is your job \
    to create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.
    make sure the questions are not repeated and check all the questions to be conforming the text as well.
    make sure to format your response like RESPONSE_JSON below and use it as guide.\
    Ensure to make {number} MCQs
    ###RESPONSE_JSON
    {response_json}
    
    
"""

In [167]:
quiz_generation_prompt=PromptTemplate(
    input_variables=["text","number","subject","tone","response_json"],
    template=TEMPLATE    
)

In [168]:
quiz_chain=LLMChain(llm=llm,prompt=quiz_generation_prompt,output_key="quiz")

In [169]:
TEMPLATE2="""
    You are an expert english grammarian and writer.Given a multiple choice quiz for {subject} students.\
    You need to evaluate complexity of the question and give a complete analysis of quiz.only use at max 50 words.
    If the quiz is not as per with the cognitive and analytical abilities of the students.\
    update the quiz questions which need to be changed and change the tone such that it perfectly fits.
    Quiz_MCQs:
    {quiz}
    
    Check from an expert english writer of the above quiz:

"""

In [170]:
callback = PromptLayerCallbackHandler()

In [171]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject","quiz"],template=TEMPLATE,callbacks=[callback])

In [172]:
review_chain=LLMChain(llm=llm,prompt=quiz_evaluation_prompt,output_key="review",verbose=True)

In [173]:
generate_evaluate_chain=SequentialChain(
    chains=[quiz_chain,review_chain],input_variables=["text","number","subject","tone","response_json"],
    output_variables=["subject","quiz"],verbose=True
)

In [174]:
with open('/home/saurabh_kokare/Music/APP/data.txt','r') as file:
    Text=file.read()

In [175]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "4": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "5": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [176]:
NUMBER=5
SUBJECT="subhash chandra bhose"
TONE='simple'

In [177]:
response=generate_evaluate_chain(
    {
        "text":Text,
        "number":NUMBER,
        "subject":SUBJECT,
        "tone":TONE,
        "response_json":json.dumps(RESPONSE_JSON)
    }
)



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

    Text:Subhas Bose followed his five brothers again 1913 to Presidency College, Calcutta, the historic and traditional college for Bengal's upper-caste Hindu men.[60][61] He chose to study philosophy, his readings including Kant, Hegel, Bergson and other Western philosophers.[62] A year earlier, he had befriended Hemanta Kumar Sarkar, a confidant and partner in religious yearnings.[63] At Presidency, their emotional ties grew stronger.[63] In the fanciful language of religious imagery, they declared their pure love for each other.[63] In the long vacations of 1914, they traveled to northern India for several months to search for a spiritual guru to guide them.[63] Subhas's family was not told clearly about the trip, leading them to think he had run away. During the trip, in which the guru proved elusive, Subhas came down with typhoid fever.[63] His absence caused emotional distress 

In [178]:
response = generate_evaluate_chain({
    "text": Text,
    "tone": "simple",
    "number":5,
    "subject": "subhash chandra bose",
    "response_json":json.dumps(RESPONSE_JSON),
})

print("Response:", response)




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

    Text:Subhas Bose followed his five brothers again 1913 to Presidency College, Calcutta, the historic and traditional college for Bengal's upper-caste Hindu men.[60][61] He chose to study philosophy, his readings including Kant, Hegel, Bergson and other Western philosophers.[62] A year earlier, he had befriended Hemanta Kumar Sarkar, a confidant and partner in religious yearnings.[63] At Presidency, their emotional ties grew stronger.[63] In the fanciful language of religious imagery, they declared their pure love for each other.[63] In the long vacations of 1914, they traveled to northern India for several months to search for a spiritual guru to guide them.[63] Subhas's family was not told clearly about the trip, leading them to think he had run away. During the trip, in which the guru proved elusive, Subhas came down with typhoid fever.[63] His absence caused emotional distress 

In [179]:
quiz=response.get("quiz")
print(quiz)

```json
{
  "1": {
    "mcq": "Why did Subhas Bose travel to Northern India during the long vacations of 1914?",
    "options": {
      "a": "To search for a spiritual guru",
      "b": "To study philosophy",
      "c": "To meet his brothers",
      "d": "To take a break from college"
    },
    "correct": "a"
  },
  "2": {
    "mcq": "What was the consequence of the incident involving Professor Oaten?",
    "options": {
      "a": "Bose was expelled from college",
      "b": "Bose's family disowned him",
      "c": "Oaten resigned from his position",
      "d": "The students were suspended"
    },
    "correct": "a"
  },
  "3": {
    "mcq": "How did Subhas Bose's family react to his expulsion?",
    "options": {
      "a": "They were anguished and upset",
      "b": "They were supportive of his actions",
      "c": "They were glad he was finally focusing on studies",
      "d": "They didn't care either way"
    },
    "correct": "a"
  },
  "4": {
    "mcq": "What did Subhas Bose study

In [180]:
print(type(quiz))

<class 'str'>


In [182]:
import pandas as pd


df = pd.DataFrame([quiz.split("|")], columns=["mcq"])